In [1]:
import fiona

In [2]:
import sys, os, time
import math

import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import shapely.geometry
import rasterio
import rasterio.features
import geopandas as gpd

def distance(pt1, pt2):
    return math.sqrt((pt1[0] - pt2[0])**2 + (pt1[1] - pt2[1])**2)

def get_side_lengths(shape):
    xs, ys = shape.boundary.xy
    pts = list(zip(xs, ys))
    lengths = []
    for i in range(len(pts)-1):
        lengths.append(distance(pts[i], pts[i+1]))
    assert len(lengths) == 4
    return sorted(lengths)

cmap = matplotlib.colors.ListedColormap([
    "#000000",
    "#FF0000"
])


Bad key "text.kerning_factor" on line 4 in
/home/jason/anaconda3/envs/ai4e/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [10]:
with rasterio.open("../landcover_scripts/all_test_inference_md/m_3807501_nw_18_1_20170609_inference.tif") as f:
    data = f.read().squeeze()
    data_transform = f.transform
    data_crs = f.crs
data.shape

(7622, 6116)

In [11]:
mask = data == 2
polygons = list(
    {'properties': {'raster_val': v}, 'geometry': s}
    for i, (s, v) in enumerate(rasterio.features.shapes(data, mask=mask, transform=data_transform))
)
len(polygons)

649

In [12]:
shape_area_threshold = 10

polygons_filtered_by_size = []
for polygon in polygons:
    shape = shapely.geometry.shape(polygon["geometry"])
        
    if shape.area > shape_area_threshold:
        polygons_filtered_by_size.append(polygon)
len(polygons_filtered_by_size)

162

In [13]:
aspect_ratio_threshold = 4

polygons_filtered_by_aspect_ratio = []
for polygon in polygons_filtered_by_size:
    shape = shapely.geometry.shape(polygon["geometry"])
    
    side_lengths = get_side_lengths(shape.minimum_rotated_rectangle)    
    short_length = min(side_lengths)
    long_length = max(side_lengths)
    
    aspect_ratio = long_length / short_length
    if aspect_ratio > aspect_ratio_threshold:
        polygons_filtered_by_aspect_ratio.append(polygon)
len(polygons_filtered_by_aspect_ratio)

17

is their a way to plot all polygonns

In [14]:
with fiona.open(
    "md_test.geojson", 'w',
    driver="GeoJSON",
    crs=f.crs,
    schema={'properties': [('raster_val', 'int')], 'geometry': 'Polygon'}
) as dst:
    dst.writerecords(polygons_filtered_by_aspect_ratio)